<a href="https://colab.research.google.com/github/pete8751/Dogbreed-Detector/blob/main/DogBreed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the tensorflow_datasets library.

In [1]:
from tensorflow.keras import layers, models, preprocessing, applications
import numpy as np
import tensorflow as tf
# from keras.src.utils import to_categorical
# import tensorflow_datasets as tfds
# import matplotlib.pyplot as plt
# import cv2

Loading in the stanford_dogs dataset, and splitting into train and test dataset. Train and Test dataset has images and labels.

In [2]:
dataset, metadata = tfds.load('stanford_dogs', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# trainData = list(train_dataset.as_numpy_iterator())
# print(trainData)
print(test_dataset)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/12000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete99T6XS/stanford_dogs-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/8580 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete99T6XS/stanford_dogs-test.tfrecord*...:   0%…

Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


Extracting metadata information: class names, number of training and test data.

In [3]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
class_names = metadata.features['label'].names

np.save('/content/class_names.npy', class_names)

# print(training_labels)
print(num_train_examples)
print(num_test_examples)
print(class_names)

12000
8580
['n02085620-chihuahua', 'n02085782-japanese_spaniel', 'n02085936-maltese_dog', 'n02086079-pekinese', 'n02086240-shih-tzu', 'n02086646-blenheim_spaniel', 'n02086910-papillon', 'n02087046-toy_terrier', 'n02087394-rhodesian_ridgeback', 'n02088094-afghan_hound', 'n02088238-basset', 'n02088364-beagle', 'n02088466-bloodhound', 'n02088632-bluetick', 'n02089078-black-and-tan_coonhound', 'n02089867-walker_hound', 'n02089973-english_foxhound', 'n02090379-redbone', 'n02090622-borzoi', 'n02090721-irish_wolfhound', 'n02091032-italian_greyhound', 'n02091134-whippet', 'n02091244-ibizan_hound', 'n02091467-norwegian_elkhound', 'n02091635-otterhound', 'n02091831-saluki', 'n02092002-scottish_deerhound', 'n02092339-weimaraner', 'n02093256-staffordshire_bullterrier', 'n02093428-american_staffordshire_terrier', 'n02093647-bedlington_terrier', 'n02093754-border_terrier', 'n02093859-kerry_blue_terrier', 'n02093991-irish_terrier', 'n02094114-norfolk_terrier', 'n02094258-norwich_terrier', 'n02094433-

Extracting data, and putting it into python lists of labels and image data.

In [ ]:
#Helper function extracting images, and labels, and resizing images, then returning list of labels and images.

def extractData(dataset):
    image_list = []
    label_list = []

    for batch_data in dataset.batch(batch_size=1).as_numpy_iterator():
        image, label = batch_data
        img = cv2.resize(image[0], target_shape, interpolation=cv2.INTER_AREA)
        # img2 = applications.xception.preprocess_input(img)
        # print(img2.shape)
        image_list.append(img)
        label_list.append(label[0])

    return (image_list, label_list)

# Convert the lists to NumPy arrays
target_shape = (299, 299)

training_images_array, training_labels_array = extractData(train_dataset)
testing_images_array, testing_labels_array = extractData(test_dataset)

Splitting into training, validation, and testing data and labels.

In [5]:
#I wanted to add an extra 1000 images for training, so I removed these from validation, and added them to training.
#I give 4000 images for validation, and around 3500 images for testing.

extra_training, extra_training_labels = testing_images_array[:1000], testing_labels_array[:1000]
training_images_array = training_images_array + extra_training
training_labels_array = training_labels_array + extra_training_labels

validation_images_array, validation_labels_array = testing_images_array[1000:5000], testing_labels_array[1000:5000]
testing_images_array, testing_labels_array = testing_images_array[5000:], testing_labels_array[5000:]

Saving Raw Data for easier access.

In [10]:
# training_images_array, validation_images_array = np.array(training_images_array), np.array(validation_images_array)
# print(training_images_array.shape)
# print(validation_images_array.shape)
# training_images_array, validation_images_array = 0, 0
testing_images_array = np.array(testing_images_array)

# np.save('/content/raw_training.npy', training_images_array)
# np.save('/content/raw_validation.npy', validation_images_array)
np.save('/content/raw_testing.npy', testing_images_array)

Saving Labels Data for easier access.

In [7]:
training_labels_array, validation_labels_array, testing_labels_array = np.array(training_labels_array), np.array(validation_labels_array), np.array(testing_labels_array)

np.save('/content/training_labels.npy', training_labels_array)
np.save('/content/validation_labels.npy', validation_labels_array)
np.save('/content/testing_labels.npy', testing_labels_array)

Raw Data and Labels Retrieval

In [2]:
# training_images_array = np.load('/content/raw_training.npy')
# validation_images_array = np.load('/content/raw_validation.npy')
# testing_images_array = np.load('/content/raw_testing.npy')

training_labels = np.load('/content/training_labels.npy')
validation_labels = np.load('/content/validation_labels.npy')
# testing_labels = np.load('/content/testing_labels.npy')

Storing validation and training data for generators.

In [9]:
i = 0
for img in training_images_array:
  padded_number = f'{i:05}'
  np.save(f'/content/training_data/{padded_number}', img)
  i += 1

i = 0
for img in validation_images_array:
  padded_number = f'{i:04}'
  np.save(f'/content/validation_data/{padded_number}', img)
  i += 1


Code to preprocess Data, and store them back in the directories.

In [12]:
# for i in range(0, len(training_labels)):
#   padded_number = padded_number = f'{i:05}'
#   npimg = np.load(f'/content/training_data/{padded_number}.npy')
#   processed = applications.inception_v3.preprocess_input(npimg)
#   np.save(f'/content/training_data/{padded_number}', processed)

# for i in range(0, len(validation_labels)):
#   padded_number = padded_number = f'{i:04}'
#   npimg = np.load(f'/content/validation_data/{padded_number}.npy')
#   processed = applications.inception_v3.preprocess_input(npimg)
#   np.save(f'/content/validation_data/{padded_number}', processed)
# import os
# os.system('rm -rf /content/training_data/*')
# os.system('rm -rf /content/validation_data/*')
devices = tf.config.experimental.list_physical_devices()
for device in devices:
  print(device.name)

/physical_device:CPU:0
/physical_device:GPU:0


Function to process data in batches and return result (conserves ram).

In [ ]:
# def batch_processer(image_array, batch_size):
#   num_batches = (len(image_array) + batch_size - 1) // batch_size
#   final_preprocessed_data = np.empty((0, 299, 299, 3))

#   for i in range(num_batches):
#     startind = i * batch_size
#     endind = min((i + 1) * batch_size, len(image_array))
#     batch_images = image_array[startind:endind]

#     preprocessed_batch = applications.xception.preprocess_input(batch_images)

#     final_preprocessed_data = np.concatenate((final_preprocessed_data, preprocessed_batch), axis=0)

#   return final_preprocessed_data

Processing Data, and saving the processed data,

In [ ]:
with tf.device('/gpu:0'):
  def batch_processer(image_array, batch_size):
    num_batches = (len(image_array) + batch_size - 1) // batch_size
    final_preprocessed_data = []

    for i in range(num_batches):
      startind = i * batch_size
      endind = min((i + 1) * batch_size, len(image_array))
      batch_images = image_array[startind:endind]

      preprocessed_batch = applications.xception.preprocess_input(batch_images)

      final_preprocessed_data.append(preprocessed_batch)

    return final_preprocessed_data

  processed_training_images = batch_processer(training_images_array, 32)
  processed_validation_images = batch_processer(validation_images_array, 32)

  np.save('/content/processed_training.npy', processed_training_images)
  np.save('/content/processed_testing.npy', processed_validation_images)

Converting to nparrays. Finding Mean and Standard Deviation, and then normalizing training and testing image arrays.

In [ ]:
# training_images_array, testing_images_array = np.array(training_images_array), np.array(testing_images_array)

# Calculating means and standard deviations for each feature.
train_mean, train_std = np.mean(training_images_array), np.std(training_images_array)
test_mean, test_std = np.mean(testing_images_array), np.std(testing_images_array)

# Normalizing. I add epsilon incase std is exactly 0.
epsilon = 0.0001
training_norm_data = (training_images_array - train_mean) / (train_std + epsilon)
testing_norm_data = (testing_images_array - test_mean) / (test_std + epsilon)

Saving processed data for quicker access later.

In [ ]:
# np.save('/content/train_dataset.npy', training_norm_data)
np.save('/content/train_labels.npy', training_labels_array)
# np.save('/content/testing_dataset.npy', testing_norm_data)
np.save('/content/testing_labels.npy', testing_labels_array)

Retrieving Personally processed Data.

In [ ]:
# print(training_norm_data[0])
# print(training_norm_data.shape)

# training_norm_data = np.load('/content/train_dataset.npy')
# training_labels_array = np.load('/content/train_labels.npy')
# testing_norm_data = np.load('/content/testing_dataset.npy')
# testing_labels_array = np.load('/content/testing_labels.npy')
# class_names = np.load('/content/class_names.npy')
# print(training_norm_data[0])
# print(training_norm_data.shape)

Retrieving data preprocessed for pre-trained model.

In [ ]:
# training_norm_data = np.load('/content/processed_training.npy')
training_norm_data = training_images_array
training_labels_array = np.load('/content/train_labels.npy')
# testing_norm_data = np.load('/content/processed_testing.npy')
testing_norm_data = testing_images_array
testing_labels_array = np.load('/content/testing_labels.npy')
class_names = np.load('/content/class_names.npy')

Splitting Testing images and labels into validation and test set. Converting labels lists into numpy arrays.

In [ ]:
# # training_labels_array, testing_labels_array = np.array(training_labels_array), np.array(testing_labels_array)
# print(training_norm_data.shape)
# print(testing_norm_data.shape)
# extra_training, extra_training_labels = testing_norm_data[:3000], testing_labels_array[:3000]
# print(extra_training.shape, extra_training_labels.shape)
# # test = np.concatenate((training_images_array, extra_training), axis=0)
# # print(test.shape)
# valarray, valabels = testing_norm_data[7000:], testing_labels_array[7000:]
# test2 = print(valarray.shape, valabels.shape)
print(training_labels_array.shape)

(15000,)


Redistributing training/testing data, and converting arrays to nparrays.

In [ ]:
# training_labels_array, testing_labels_array = np.array(training_labels_array), np.array(testing_labels_array)

# #I wanted to add an extra 3000 images for training, so I removed these from validation, and added them to training.
# #I give 3000 images for validation, and around 1500 images for testing.

# extra_training, extra_training_labels = testing_norm_data[:3000], testing_labels_array[:3000]
# training_norm_data = np.concatenate((training_norm_data, extra_training), axis=0)
# training_labels_array = np.concatenate((training_labels_array, extra_training_labels), axis=0)

# print(training_norm_data.shape)
# validation_images_array, validation_labels_array = testing_norm_data[3000:7000], testing_labels_array[3000:7000]
# testing_norm_data, testing_labels_array = testing_norm_data[7000:], testing_labels_array[7000:]

Displaying some test images with labels.

In [ ]:
def display_image(image_array):
    plt.imshow(image_array)
    plt.axis('off')
    plt.show()

image = validation_images_array[0]
# print(image.shape)

print(validation_images_array.shape)

image_test = testing_norm_data[0]

# Comparing Training images and label list.
display_image(image)
print(class_names[validation_labels_array[0]])

#Comparing testing images and label list
display_image(image_test)
print(class_names[testing_labels_array[0]])

AttributeError: 'list' object has no attribute 'shape'

Training the Model.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(120, activation='softmax'))

Model Fitting and Compiling

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_norm_data, training_labels_array, epochs=10, validation_data=(validation_images_array, validation_labels_array))

Implementing the generator method to reduce overfitting.

In [ ]:
# Creating the train_generator
train_datagen = preprocessing.image.ImageDataGenerator(
        # rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,)

# Creating the validation_generator
validation_datagen = preprocessing.image.ImageDataGenerator(
        # rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,)
with tf.device('/gpu:0'):
  train_generator = train_datagen.flow(training_images_array, training_labels, batch_size=32, shuffle=True)
  validation_generator = validation_datagen.flow(validation_images_array, validation_labels, batch_size = 32, shuffle=True)

Generator from directory.

In [3]:
def custom_data_generator(file_paths, labels, batch_size=32, shuffle=False, drop_remainder=False):

    num_samples = len(file_paths)
    steps_per_epoch = (num_samples + batch_size - 1) // batch_size

    while True:
        for i in range(steps_per_epoch):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, num_samples)
            batch_file_paths = file_paths[start_idx:end_idx]
            batch_labels = labels[start_idx:end_idx]

            batch_images = [process_file_path(file_path) for file_path in batch_file_paths]
            yield np.array(batch_images), batch_labels

def process_file_path(file_path):
  npimg = np.load(f'{file_path}')
  # return npimg
  return applications.inception_v3.preprocess_input(npimg)
  # return applications.ception.preprocess_input(npimg)


# Example: Using the custom generator
# file_paths_training = [f'/content/training_data/{i:05}.npy' for i in range(0, 15000)]
# labels_array = training_labels_array

# generator = custom_data_generator(file_paths_training, labels_array)

Making validation, and training generators from the above generator.

In [4]:
file_paths_training = [f'/content/training_data/{i:05}.npy' for i in range(0, len(training_labels))]
print(len(file_paths_training))
file_paths_validation = [f'/content/validation_data/{i:04}.npy' for i in range(0, len(validation_labels))]

training_generator = custom_data_generator(file_paths_training, training_labels)
validation_generator = custom_data_generator(file_paths_validation, validation_labels)
# training_images_array = 0
# validation_images_array = 0
# testing_images_array = 0

# for _ in range(5):  # Replace 5 with the desired number of steps
#     images, labels = next(validation_generator)
#     print(images.shape)
#     print(labels.shape)
#     # Now 'images' and 'labels' contain the data for the current batch

13000


Building the new Model.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(100, 100, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120))
model.add(layers.Activation('sigmoid'))

Setting up a checkpoint

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = '/content/dogbreed_best_model_light'

checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',  # or 'val_accuracy' for accuracy
                             save_best_only=True,
                             mode='min',  # or 'max' for accuracy
                             verbose=1)

Compiling and Running the Model.

In [10]:
import pickle

# Assuming 'history' is the result of model.fit
history_path = "training_history.pkl"

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# print(validation_images_array.shape)
# print(validation_labels_array.shape)
#now we can fit the model
with tf.device('/gpu:0'):
  history = model.fit(training_generator, epochs=10, steps_per_epoch = 407, validation_data=validation_generator, validation_steps = 125, callbacks=[checkpoint])

# Save history to file
with open('/content/history', 'wb') as file:
    pickle.dump(history.history, file)

Epoch 1/10
407/407 [==============================] - ETA: 0s - loss: 1.0534 - accuracy: 0.7577
Epoch 1: val_loss improved from 6.78150 to 0.45015, saving model to /content/dogbreed_best_model_light
407/407 [==============================] - 163s 382ms/step - loss: 1.0534 - accuracy: 0.7577 - val_loss: 0.4502 - val_accuracy: 0.8575
Epoch 2/10
407/407 [==============================] - ETA: 0s - loss: 0.2945 - accuracy: 0.9146
Epoch 2: val_loss did not improve from 0.45015
407/407 [==============================] - 147s 360ms/step - loss: 0.2945 - accuracy: 0.9146 - val_loss: 0.4526 - val_accuracy: 0.8640
Epoch 3/10
407/407 [==============================] - ETA: 0s - loss: 0.1573 - accuracy: 0.9559
Epoch 3: val_loss improved from 0.45015 to 0.43348, saving model to /content/dogbreed_best_model_light
407/407 [==============================] - 165s 405ms/step - loss: 0.1573 - accuracy: 0.9559 - val_loss: 0.4335 - val_accuracy: 0.8777
Epoch 4/10
407/407 [==============================] - 

Building the Xception Model.

In [9]:
base_model = applications.Xception(
    input_shape=(299, 299, 3),
    include_top=False,
    weights="imagenet",
    pooling=None,  # Global average pooling for flattening
)

model = models.Sequential()
model.add(base_model)
model.layers[0].trainable = False
# model.add(layers.Flatten())
# model.add(layers.Dense(120, activation='softmax'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.BatchNormalization())
model.add(layers.Dense(120, activation='softmax'))

Building the Inception Model.

In [5]:
base_model = applications.InceptionV3(
    input_shape=(299, 299, 3),
    include_top=False,
    weights="imagenet",
    pooling=None,
)

model = models.Sequential()
model.add(base_model)
model.layers[0].trainable = False
model.add(layers.Flatten())
model.add(layers.Dense(120, activation='softmax'))
# model.add(layers.Flatten(input_shape=train_features.shape[1:]))

Determining and saving set of features of pre-trained model for training data.

In [10]:
# pre_trained_validation = model.predict(validation_images_array)
# np.save('/content/pre_trained_validation.npy', pre_trained_validation)
with tf.device('/gpu:0'):
  pre_trained_data = model.predict(training_generator, steps = 1300)

np.save('/content/pre_trained_validation.npy', pre_trained_data)
# np.save('/content/pre_trained_bottleneck_features_inceptionv3.npy', model.predict(training_generator, steps = 469))

1300/1300 [==============================] - 57s 44ms/step


ResourceExhaustedError: {{function_node __wrapped__ConcatV2_N_1300_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[13000,8,8,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ConcatV2] name: concat

Determining and saving set of features of pre-trained model for Validation data.

In [7]:
# pre_trained_validation = model.predict(validation_generator)
np.save('/content/pre_trained_validation_inceptionv3.npy', model.predict(validation_generator, steps = 125))

NameError: name 'validation_generator' is not defined

In [ ]:
train_features = np.load('/content/pre_trained_bottleneck_features_inceptionv3.npy')
validation_features = np.load('/content/pre_trained_validation_inceptionv3.npy')
print(train_features.shape)

(15000, 1, 1, 2048)


In [ ]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=train_features.shape[1:]))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(512, activation='softmax'))
model.add(layers.Dense(120, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_features, training_labels_array, epochs=10, validation_data=(validation_features, validation_labels_array))
# model.save('/content/DogBreed Models')

Epoch 1/10
469/469 [==============================] - 5s 7ms/step - loss: 4.4626 - accuracy: 0.1596 - val_loss: 3.8669 - val_accuracy: 0.2325
Epoch 2/10
469/469 [==============================] - 4s 9ms/step - loss: 2.9866 - accuracy: 0.3265 - val_loss: 3.9034 - val_accuracy: 0.2333
Epoch 3/10
469/469 [==============================] - 3s 6ms/step - loss: 2.3787 - accuracy: 0.4233 - val_loss: 4.0871 - val_accuracy: 0.2370
Epoch 4/10
469/469 [==============================] - 3s 6ms/step - loss: 1.9363 - accuracy: 0.5046 - val_loss: 4.1455 - val_accuracy: 0.2395
Epoch 5/10
469/469 [==============================] - 3s 6ms/step - loss: 1.6424 - accuracy: 0.5675 - val_loss: 4.2614 - val_accuracy: 0.2377
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 1.3682 - accuracy: 0.6351 - val_loss: 4.5281 - val_accuracy: 0.2280
Epoch 7/10
469/469 [==============================] - 4s 8ms/step - loss: 1.1886 - accuracy: 0.6787 - val_loss: 4.7053 - val_accuracy: 0.2315
Epoch 

Code to summarize and save models.

In [ ]:
# train_data = np.load(open('/content/pre_trained_output_headless.npy', 'rb'))
print(train_data.shape[1:])
# model.summary()
# model.save('/content/DogBreed Models')

(3, 3, 2048)


Code to save models to my google drive.

In [12]:
from google.colab import drive
import shutil

# drive.mount('/content/gdrive', force_remount=True)
source_folder = '/content/history'  # Replace with the path to your source folder
destination_folder = '/content/gdrive/MyDrive/breed_models/history_full'  # Replace with the path to your destination folder in Google Drive

shutil.copytree(source_folder, destination_folder)

NotADirectoryError: [Errno 20] Not a directory: '/content/history'